In [1]:
import pandas as pd
from sunpy.coordinates import HeliographicStonyhurst, Helioprojective
import astropy.units as u
from astropy.coordinates import SkyCoord
from sunpy.net import Fido
from sunpy.net import attrs as a
from sunpy.map import Map
from sunpy.coordinates import frames, get_body_heliographic_stonyhurst, get_horizons_coord, transform_with_sun_center
import matplotlib.pyplot as plt

In [2]:
df1 = pd.read_csv('flare_location_output4_part1.csv')
df2 = pd.read_csv('flare_location_output4_part2.csv')

In [6]:
df1.head()

,flare ID,request ID,X flare loc,Y flare loc,FWDFIT X SIGMA,FWDFIT Y SIGMA,MAX X COORD BP,MAX Y COORD BP
0,2211090050,2211090305,-15.344300,181.459000,0.918372,1.007100,-16.415800,180.574000
1,2211090458,2211094607,757.925903,-921.147644,13.152131,6.636548,752.496948,-924.262573
2,2211090551,2211090238,-758.126221,1789.811890,5.702117,12.557203,-751.933289,1781.755005
3,2211090720,2211091049,359.304382,523.129578,0.484560,0.617333,359.168732,522.427246
4,2211090908,2211096407,377.920044,529.196899,1.762542,2.026066,374.929199,529.791260


In [9]:
len(df1), len(df2)

(73, 73)

In [7]:
df2.head()

,flare ID,STIX MAX X BP COORD,STIX MAX Y BP COORD,time shift
0,2211090050,97.614000,35.544700,198.206746
1,2211090458,-992.386414,-757.579285,197.137285
2,2211090551,1690.018311,790.110046,196.896927
3,2211090720,-202.187622,-1784.592041,196.524467
4,2211090908,-193.186462,-1797.633179,196.084910


In [4]:
new_df = df2.join(df1.set_index('flare ID'), on='flare ID', how='right')

In [8]:
len(new_df)

73

In [10]:
full_fl_paths = pd.read_csv('full_flarelist_with_paths.csv')

In [11]:
full_fl_paths.head()

,flare_id,att_in,GOES_flux,start_UTC,peak_UTC,end_UTC,4-10 keV,10-15 keV,number_available_files,available_file_request_IDs,solo_lat,solo_lon,solo_AU_distance,15-25 keV,25-50 keV,50-84 keV,GOES_class,Request IDs,aux_paths,pixel_paths
0,2211090050,False,6.073233e-07,2022-11-09T00:44:44.012,2022-11-09 00:50:20.013,2022-11-09T00:52:56.013,1215,271,1,[2211090305],9.180063,-24.156920,0.590873,135,799,463,B6.1,2211090305,./aux_data/solo_L2_stix-aux-ephemeris_20221109...,./pixel_data/solo_L1_stix-sci-xray-cpd_2022110...
1,2211090458,False,6.310129e-07,2022-11-09T04:52:52.037,2022-11-09 04:58:16.038,2022-11-09T05:06:20.039,335,75,1,[2211094607],9.177897,-23.999941,0.592948,83,863,431,B6.3,2211094607,./aux_data/solo_L2_stix-aux-ephemeris_20221109...,./pixel_data/solo_L1_stix-sci-xray-cpd_2022110...
2,2211090551,False,7.278045e-07,2022-11-09T05:39:48.042,2022-11-09 05:51:40.043,2022-11-09T06:12:08.045,367,67,1,[2211090238],9.177393,-23.966433,0.593394,83,863,463,B7.3,2211090238,./aux_data/solo_L2_stix-aux-ephemeris_20221109...,./pixel_data/solo_L1_stix-sci-xray-cpd_2022110...
3,2211090720,False,9.645386e-07,2022-11-09T07:18:12.052,2022-11-09 07:20:44.052,2022-11-09T07:28:40.053,2431,247,1,[2211091049],9.176525,-23.910781,0.594137,115,799,463,B9.6,2211091049,./aux_data/solo_L2_stix-aux-ephemeris_20221109...,./pixel_data/solo_L1_stix-sci-xray-cpd_2022110...
4,2211090908,False,6.582638e-07,2022-11-09T08:58:40.062,2022-11-09 09:08:44.063,2022-11-09T09:14:56.064,607,135,1,[2211096407],9.175424,-23.843695,0.595037,107,863,463,B6.6,2211096407,./aux_data/solo_L2_stix-aux-ephemeris_20221109...,./pixel_data/solo_L1_stix-sci-xray-cpd_2022110...


In [12]:
final_df = pd.merge(full_fl_paths, new_df, right_on ='flare ID', left_on = 'flare_id')

In [14]:
len(final_df)

73

In [15]:
final_df.head()

,flare_id,att_in,GOES_flux,start_UTC,peak_UTC,end_UTC,4-10 keV,10-15 keV,number_available_files,available_file_request_IDs,...,STIX MAX X BP COORD,STIX MAX Y BP COORD,time shift,request ID,X flare loc,Y flare loc,FWDFIT X SIGMA,FWDFIT Y SIGMA,MAX X COORD BP,MAX Y COORD BP
0,2211090050,False,6.073233e-07,2022-11-09T00:44:44.012,2022-11-09 00:50:20.013,2022-11-09T00:52:56.013,1215,271,1,[2211090305],...,97.614000,35.544700,198.206746,2211090305,-15.344300,181.459000,0.918372,1.007100,-16.415800,180.574000
1,2211090458,False,6.310129e-07,2022-11-09T04:52:52.037,2022-11-09 04:58:16.038,2022-11-09T05:06:20.039,335,75,1,[2211094607],...,-992.386414,-757.579285,197.137285,2211094607,757.925903,-921.147644,13.152131,6.636548,752.496948,-924.262573
2,2211090551,False,7.278045e-07,2022-11-09T05:39:48.042,2022-11-09 05:51:40.043,2022-11-09T06:12:08.045,367,67,1,[2211090238],...,1690.018311,790.110046,196.896927,2211090238,-758.126221,1789.811890,5.702117,12.557203,-751.933289,1781.755005
3,2211090720,False,9.645386e-07,2022-11-09T07:18:12.052,2022-11-09 07:20:44.052,2022-11-09T07:28:40.053,2431,247,1,[2211091049],...,-202.187622,-1784.592041,196.524467,2211091049,359.304382,523.129578,0.484560,0.617333,359.168732,522.427246
4,2211090908,False,6.582638e-07,2022-11-09T08:58:40.062,2022-11-09 09:08:44.063,2022-11-09T09:14:56.064,607,135,1,[2211096407],...,-193.186462,-1797.633179,196.084910,2211096407,377.920044,529.196899,1.762542,2.026066,374.929199,529.791260


In [13]:
final_df.keys()

Index(['flare_id', 'att_in', 'GOES_flux', 'start_UTC', 'peak_UTC', 'end_UTC',
       '4-10 keV', '10-15 keV', 'number_available_files',
       'available_file_request_IDs', 'solo_lat', 'solo_lon',
       'solo_AU_distance', '15-25 keV', '25-50 keV', '50-84 keV', 'GOES_class',
       'Request IDs', 'aux_paths', 'pixel_paths', 'flare ID',
       'STIX MAX X BP COORD', 'STIX MAX Y BP COORD', 'time shift',
       'request ID', 'X flare loc', 'Y flare loc', 'FWDFIT X SIGMA',
       'FWDFIT Y SIGMA', 'MAX X COORD BP', 'MAX Y COORD BP'],
      dtype='object')

In [16]:
earth_x = []
earth_y = []
for i in range(len(final_df)):
    x = final_df.iloc[i]['X flare loc']
    y = final_df.iloc[i]['Y flare loc']
    obstime = final_df.iloc[i]['peak_UTC'] 
    solo_pos = get_horizons_coord("solar orbiter",obstime)
    earth_coord = SkyCoord(x*u.arcsec, y*u.arcsec, frame=Helioprojective, observer=solo_pos).transform_to(Helioprojective(observer="earth", obstime=obstime))
    earth_x.append(earth_coord.Tx.value)
    earth_y.append(earth_coord.Ty.value)
    

INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]


INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecra

In [17]:
final_df["Earth X"] = earth_x 
final_df['Earth Y'] = earth_y

In [18]:
final_df.keys()

Index(['flare_id', 'att_in', 'GOES_flux', 'start_UTC', 'peak_UTC', 'end_UTC',
       '4-10 keV', '10-15 keV', 'number_available_files',
       'available_file_request_IDs', 'solo_lat', 'solo_lon',
       'solo_AU_distance', '15-25 keV', '25-50 keV', '50-84 keV', 'GOES_class',
       'Request IDs', 'aux_paths', 'pixel_paths', 'flare ID',
       'STIX MAX X BP COORD', 'STIX MAX Y BP COORD', 'time shift',
       'request ID', 'X flare loc', 'Y flare loc', 'FWDFIT X SIGMA',
       'FWDFIT Y SIGMA', 'MAX X COORD BP', 'MAX Y COORD BP', 'Earth X',
       'Earth Y'],
      dtype='object')

In [19]:
filtered = final_df[final_df['4-10 keV']>5*10**2]

In [20]:
final_df.to_csv('091122-121122_final_full_flare_list.csv')

In [21]:
filtered.to_csv('091122-121122_final_filtered_flare_list.csv')

In [16]:
len(final_df)

47

In [17]:
len(filtered)

41

In [126]:
filtered

,flare_id,att_in,GOES_class,GOES_flux,start_UTC,peak_UTC,end_UTC,4-10 keV,10-15 keV,15-25 keV,...,time shift,request ID,X flare loc,Y flare loc,FWDFIT X SIGMA,FWDFIT Y SIGMA,MAX X COORD BP,MAX Y COORD BP,Earth X,Earth Y
0,2211090050,False,B6.1,6.073233e-07,2022-11-09T00:44:44.012,2022-11-09 00:50:20.013,2022-11-09T00:52:56.013,1215,271,135,...,198.206746,2211090305,-15.344000,181.460000,1.378700,0.866219,-16.415800,180.574000,-382.781073,187.116582
4,2211091554,False,B7.1,7.068386e-07,2022-11-09T15:50:44.105,2022-11-09 15:54:32.105,2022-11-09T16:06:36.107,1087,135,91,...,194.340023,2211099586,148.352203,170.061981,2.744007,2.867299,145.886475,162.096085,-282.569885,179.599161
6,2211091958,False,C4.4,4.366502e-06,2022-11-09T19:34:00.128,2022-11-09 19:58:36.130,2022-11-09T23:16:24.150,8703,495,231,...,193.301711,2211092397,-854.139832,-920.446228,0.505462,0.338237,-856.228760,-920.849854,-737.787501,-495.122851
7,2211100208,False,B6.1,6.083465e-07,2022-11-10T02:06:48.168,2022-11-10 02:08:36.168,2022-11-10T02:17:00.169,1087,271,91,...,191.768032,2211108453,261.750977,195.509872,4.143073,8.529701,249.142883,192.884811,-207.419107,193.880308
10,2211100844,False,B8.1,8.130161e-07,2022-11-10T08:40:36.208,2022-11-10 08:44:52.209,2022-11-10T08:48:08.209,1087,167,75,...,190.115018,2211106426,654.651917,481.418579,0.863806,0.852569,647.502808,479.631714,73.427288,354.625780
17,2211102118,False,B7.1,7.122287e-07,2022-11-10T21:14:32.086,2022-11-10 21:18:40.086,2022-11-10T21:21:24.086,1087,199,115,...,186.983428,2211100333,677.836548,146.374527,2.227091,1.037061,672.683044,142.450531,72.315416,154.034864
22,2211110040,False,B9.8,9.795699e-07,2022-11-11T00:20:08.105,2022-11-11 00:40:20.107,2022-11-11T00:47:56.108,1343,231,91,...,186.173245,2211110188,723.065918,139.364075,1.047883,0.760175,718.256348,134.179764,106.313993,148.306078
23,2211110313,False,C2.5,2.501505e-06,2022-11-11T02:42:40.119,2022-11-11 03:13:36.122,2022-11-11T03:45:32.026,23551,6399,4351,...,185.507042,2211110829,738.871155,145.420746,0.484500,0.270436,732.570007,141.787735,120.088304,151.647620
24,2211110437,False,C3.3,3.313714e-06,2022-11-11T04:30:48.030,2022-11-11 04:37:16.031,2022-11-11T04:59:16.033,29695,3455,991,...,185.162390,2211117374,758.166809,139.450699,0.338996,0.182153,755.373596,133.764069,134.681212,147.305662
25,2211110510,False,B7.8,7.821267e-07,2022-11-11T05:06:12.034,2022-11-11 05:10:24.034,2022-11-11T05:14:16.035,1343,367,247,...,185.035755,2211113005,751.728271,148.935181,1.265695,1.180821,747.182007,141.571320,131.165485,153.479435


# Transformation sanity check for known event


In [129]:
solo_pos_march22 = get_horizons_coord("solar orbiter", "2022-03-30")


INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]


In [130]:
solo_pos_march22

<SkyCoord (HeliographicStonyhurst: obstime=2022-03-30T00:00:00.000, rsun=695700.0 km): (lon, lat, radius) in (deg, deg, AU)
    (91.74058012, 1.30847464, 0.33146424)>

In [137]:
earth_coord_march22 = SkyCoord(-2485*u.arcsec, 600*u.arcsec, frame=Helioprojective, observer=solo_pos_march22).transform_to(Helioprojective(observer="earth", obstime='2022-03-30'))


In [138]:
earth_coord_march22

<SkyCoord (Helioprojective: obstime=2022-03-30T00:00:00.000, rsun=695700.0 km, observer=<HeliographicStonyhurst Coordinate for 'earth'>): (Tx, Ty, distance) in (arcsec, arcsec, AU)
    (482.59909625, 301.44357158, 0.99477816)>